Notebook to do RHF-DMRG of H2 for test purposes

In [2]:
from pyscf import gto, scf, mcscf
import numpy as np
import matplotlib.pyplot as plt
from pyblock2._pyscf.ao2mo import integrals as itg
from pyblock2.driver.core import DMRGDriver, SymmetryTypes
import os 

In [7]:
# to delete the tmp files

folder_path = "./tmp"

for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        
        # Check if it's a file (not a directory) and delete it
        if os.path.isfile(file_path):
            os.remove(file_path)
            print(f"Deleted: {file_path}")

Deleted: ./tmp/F.MPS.GS.0
Deleted: ./tmp/F.MPS.GS.1
Deleted: ./tmp/F.MPS.GS.2
Deleted: ./tmp/F.MPS.INFO.GS.LEFT.0
Deleted: ./tmp/F.MPS.INFO.GS.RIGHT.0
Deleted: ./tmp/F.MPS.INFO.GS.LEFT.1
Deleted: ./tmp/F.MPS.INFO.GS.RIGHT.1
Deleted: ./tmp/F.MPS.INFO.GS.LEFT.2
Deleted: ./tmp/F.MPS.INFO.GS.RIGHT.2
Deleted: ./tmp/F.MPS.INFO.GS.LEFT.3
Deleted: ./tmp/F.MPS.INFO.GS.RIGHT.3
Deleted: ./tmp/F.MPS.GS.-1
Deleted: ./tmp/GS-mps_info.bin


In [8]:
#RHF-DMRG
H2 = gto.M(atom=[["H", 0, 0, 0],
                  ["H", 1.57, 0, 0]], basis = 'sto-3g', spin = 0)
rhf=scf.RHF(H2).run()

norbs, nelecas = (2, 2)
ref_ucas = mcscf.CASSCF(rhf, norbs, nelecas).mc1step()[0]


bond_dims = [10] * 10
noises = [1e-4] *  5 + [1e-5] * 5 + [0]
thrds = [1e-8] * 10
ncas, n_elec, spin, ecore, h1e, g2e, orb_sym = itg.get_rhf_integrals(rhf,
    ncore=0, ncas=None, g2e_symm=1)

driver = DMRGDriver(scratch="./tmp", symm_type=SymmetryTypes.SU2, n_threads=4)
driver.initialize_system(n_sites=ncas, n_elec=n_elec, spin=spin, orb_sym=orb_sym)

mpo = driver.get_qc_mpo(h1e=h1e, g2e=g2e, ecore=ecore, iprint=1)
ket = driver.get_random_mps(tag="GS", bond_dim=10, nroots=1)
energy = driver.dmrg(mpo, ket, n_sweeps=15, bond_dims=bond_dims, noises=noises,
    thrds=thrds, iprint=1)
print('DMRG energy = %20.15f' % energy)

converged SCF energy = -0.890301503470727
CASSCF energy = -0.987594783712069
CASCI E = -0.987594783712069  E(CI) = -1.32465033206876  S^2 = 0.0000000


integral symmetrize error =  0.0
integral cutoff error =  0.0
mpo terms =         22

Build MPO | Nsites =     2 | Nterms =         22 | Algorithm = FastBIP | Cutoff = 1.00e-20
 Site =     0 /     2 .. Mmpo =    12 DW = 0.00e+00 NNZ =       12 SPT = 0.0000 Tmvc = 0.000 T = 0.010
 Site =     1 /     2 .. Mmpo =     1 DW = 0.00e+00 NNZ =       12 SPT = 0.0000 Tmvc = 0.000 T = 0.001
Ttotal =      0.011 Tmvc-total = 0.000 MPO bond dimension =    12 MaxDW = 0.00e+00
NNZ =           24 SIZE =           24 SPT = 0.0000

Rank =     0 Ttotal =      0.023 MPO method = FastBipartite bond dimension =      12 NNZ =           24 SIZE =           24 SPT = 0.0000

Sweep =    0 | Direction =  forward | Bond dimension =   10 | Noise =  1.00e-04 | Dav threshold =  1.00e-08
Time elapsed =      0.027 | E =      -0.9875947837 | DW = 0.00000e+00

Sweep =    1 | Direction = backward | Bond dimension =   10 | Noise =  1.00e-04 | Dav threshold =  1.00e-08
Time elapsed =      0.031 | E =      -0.9875947837 | DE 

In [9]:
Mo_coeff=rhf.mo_coeff
print(Mo_coeff)

[[ 0.63704121  0.80690766]
 [ 0.63704121 -0.80690766]]
